In [2]:
# Root path of Fremont Dropbox
import os
import sys
import demand_util

# array analysis
import numpy as np
import osmnx as ox
import pandas as pd
import sklearn.cluster as skc
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

# geo spacial data analysis
import fiona
import geopandas as gpd
from shapely import wkt
from rtree import index
from keplergl import KeplerGl

# assorted parsing and modeling tools
import csv
import time
import math
import random
import requests
from pytz import utc
from osrm_util import *
from shutil import copyfile, copytree
from shapely.ops import nearest_points, unary_union
from shapely.geometry import box, Point, LineString, Polygon, MultiPoint, MultiPolygon, GeometryCollection

# import polyline
from pathlib import Path

# importing all the Kepler.gl configurations
import ast

from urllib.error import HTTPError

In [3]:
# updates modules when changed
%load_ext autoreload
%autoreload 2

In [4]:
project_delimitation = []
project_delimitation.append((-121.94277062699996, 37.55273259000006))
project_delimitation.append((-121.94099807399999, 37.554268507000074))
project_delimitation.append((-121.91790942699998, 37.549823434000075))
project_delimitation.append((-121.89348666299998, 37.52770136500004, ))
project_delimitation.append((-121.90056572499998, 37.52292299800007))
project_delimitation.append((-121.90817571699995, 37.52416183400004))
project_delimitation.append((-121.91252749099999, 37.51845069500007))
project_delimitation.append((-121.91349347899995, 37.513972023000065))
project_delimitation.append((-121.90855417099999, 37.503837324000074))
project_delimitation.append((-121.91358547299996, 37.50097863000008))
project_delimitation.append((-121.90798018999999, 37.49080413200005))
project_delimitation.append((-121.91894942199997, 37.48791568200005))
project_delimitation.append((-121.92029048799998, 37.488706567000065))
project_delimitation.append((-121.93070953799997, 37.48509600500006))
project_delimitation.append((-121.93254686299997, 37.48864173700008))
project_delimitation.append((-121.94079404499996, 37.50416395900004))
project_delimitation.append((-121.94569804899999, 37.51332606200003))
project_delimitation.append((-121.94918207899997, 37.520371545000046))
project_delimitation.append((-121.95305006999996, 37.52804520800004))
project_delimitation.append((-121.953966735, 37.53272020000003))
project_delimitation.append((-121.95428756799998, 37.53817435800005))
project_delimitation.append((-121.95506236799997, 37.54107322100003))
project_delimitation.append((-121.95676186899999, 37.54656695700004))
project_delimitation.append((-121.95529950799994, 37.54980786700003))
project_delimitation.append((-121.95261192399994, 37.550479763000055))
project_delimitation.append((-121.94988481799999, 37.55277211300006))
project_delimitation.append((-121.94613010599994, 37.55466923100005))
project_delimitation.append((-121.94277062699996, 37.55273259000006))

In [5]:
# Setting up the Coordinate Reference Systems up front in the necessary format.
crs_degree = {'init': 'epsg:4326'} # CGS_WGS_1984 (what the GPS uses)

# We let this notebook to know where to look for fremontdropbox module
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from fremontdropbox import get_dropbox_location
# Root path of the Dropbox business account
dbx = get_dropbox_location()

# Temporary! Location of the folder where the restructuring is currently happening
data_path = dbx + '/Private Structured data collection'

In [6]:
sfcta_folder = os.path.join(data_path, "Data processing", "Raw", "Demand", "OD demand", "SFCTA demand data")
sections_path = os.path.join(data_path, "Aimsun", "Inputs", "sections.shp")
sections_shp = gpd.GeoDataFrame.from_file(sections_path)
sections_shp = sections_shp.to_crs(crs_degree) 

C:\Users\jainc\AppData\Roaming\Python\Python37\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
def from_csv(csv_df):
    all_points = []
    node_types = ['start', 'end']
    for node_type in node_types:
        points = list(zip(csv_df[node_type + '_node_lng'], csv_df[node_type + '_node_lat']))
        all_points.extend(points)
    return all_points

def to_csv(file_name, header, lines):
    def add_quotes(val):
        return "\"" + str(val) + "\"" if ',' in str(val) else str(val)

    csv = open(file_name, 'w')
    csv.write(header + '\n')
    for line in lines:
        csv.write(','.join(map(add_quotes, line)) + '\n')

In [7]:
def multisplit_polygon(geometry, threshold=0.005, count=0):
    """
    Split a Polygon repeatedly across it's shortest dimension
    Copyright (c) 2016, Joshua Arnott
    Source: https://snorfalorpagus.net/blog/2016/03/13/splitting-large-polygons-for-faster-intersections/
    """
    bounds = geometry.bounds
    width = bounds[2] - bounds[0]
    height = bounds[3] - bounds[1]
    if max(width, height) <= threshold or count == 250:
        # either the polygon is smaller than the threshold, or the maximum
        # number of recursions has been reached
        return [geometry]
    if height >= width:
        # split left to right
        a = box(bounds[0], bounds[1], bounds[2], bounds[1]+height/2)
        b = box(bounds[0], bounds[1]+height/2, bounds[2], bounds[3])
    else:
        # split top to bottom
        a = box(bounds[0], bounds[1], bounds[0]+width/2, bounds[3])
        b = box(bounds[0]+width/2, bounds[1], bounds[2], bounds[3])
    result = []
    for d in (a, b,):
        c = geometry.intersection(d)
        if not isinstance(c, GeometryCollection):
            c = [c]
        for e in c:
            if isinstance(e, (Polygon, MultiPolygon)):
                result.extend(multisplit_polygon(e, threshold, count+1))
    if count > 0:
        return result
    # convert multipart into singlepart
    final_result = []
    for g in result:
        if isinstance(g, MultiPolygon):
            final_result.extend(g)
        else:
            final_result.append(g)
    return final_result

In [8]:
# split = multisplit_polygon(Polygon(project_delimitation), 0.005)
# split_gdf = gpd.GeoDataFrame(geometry=split)
# one_gdf = gpd.GeoDataFrame(geometry=[Polygon(project_delimitation)])
# map_2 = KeplerGl(height=1000)
# map_2.add_data(data=split_gdf, name = "Split GDF")
# map_2.add_data(data=one_gdf, name = "One GDF")
# map_2

In [9]:
from json.decoder import JSONDecodeError
def shortest_path_by_travel_time(start, end):
    """
    Using OSM to get shortest path (weighted by free-flow travel time) from start to end location
    where start and end are Point objects, and graph is the NetworkX graph of the region
    """
    try:
        query_result = simple_route(
                                  [start.x, start.y], [end.x, end.y],
                                  output='route', overview="full", geometry='wkt')
        shortest_path = wkt.loads(query_result[0]['geometry'])
        
        #print('yay')
        
        return shortest_path
    
    except JSONDecodeError as err:
        
        #print('darn')
        #print(err)
        #print([start.x, start.y], [end.x, end.y])
        
        return shortest_path_by_travel_time(start, end)    

In [10]:
def create_external_delim(dir_taz):
    ''' 
    Create a external demand delimitation:
    - load SFCTA data as Geopandas point (one point = one origin or one destination)
    - Get convex hull of the point
    - Use the convex hull (+ buffer) as the external demand delimitation
    '''
    # load the 3 csv files
    ending_csv = pd.read_csv(os.path.join(dir_taz, "ending_fremont_legs.csv"))
    internal_csv = pd.read_csv(os.path.join(dir_taz, "internal_fremont_legs.csv"))
    starting_csv = pd.read_csv(os.path.join(dir_taz, "starting_fremont_legs.csv"))

    # get the points from the csv's (start and end points)
    points = []
    points.extend(from_csv(ending_csv))
    points.extend(from_csv(internal_csv))
    points.extend(from_csv(starting_csv))
    points = np.array(points)

    # get convex hull of points
    hull = ConvexHull(points)
    hull_points = points[hull.vertices, :]

    # add buffer to convex hull
    def normalize(point):
        norm = np.linalg.norm(point)
        return point / norm if norm > 0 else point

    # for each point calculate the direction to expand for buffer
    buffer_directions = []
    for i in range(len(hull_points)):
        point = hull_points[i]
        left_neighbor = hull_points[(i-1) % len(hull_points)]
        right_neighbor = hull_points[(i+1) % len(hull_points)]
        left_arrow = point - left_neighbor
        right_arrow = point - right_neighbor
        left_arrow = normalize(left_arrow)
        right_arrow = normalize(right_arrow)
        buffer_directions.append(normalize(left_arrow + right_arrow))
    buffer_directions = np.array(buffer_directions)

    # calculate the new (expanded) hull points with buffer
    buffer_coefficient = .05
    expanded_hull_points = hull_points + buffer_coefficient * buffer_directions
    
    return expanded_hull_points

def create_external_centroids(sections_df, expanded_hull_points):
    '''
    Create external centroids:
    - select road with no fnode and capacity above 800 from sections_df
    - create a point at the end of all selected road
    - plot the points, get a list of points to remove visually
    '''
    # select roads with no fnode and capacity above 800 from sections_df
    sections_df = sections_df[pd.isnull(sections_df['fnode']) & (sections_df['capacity'] > 800)]
    sections_df = sections_df[['eid', 'geometry']]

    # filter out roads that are visually erroneous -> a road not entering the project area (Fremont)
    # sections_df.to_csv('selected_roads.csv')   # roads to remove obtained visually
    ######################################################################################
    ########################## NEED AN AUTOMATED WAY TO DO THIS ##########################
    ######################################################################################
    roads_to_remove = [56744, 30676, 35572, 56534]
    sections_df = sections_df.astype({'eid': 'int32'})
    sections_df = sections_df[~sections_df['eid'].isin(roads_to_remove)]

    # create external centroid nodes -> create a point at the terminal end of these roads
    # that is, for each road find the end of the road that is closer to the external delimitation (convex hull)
    external_centroid_nodes = []
    internal_centroid_nodes = []  # need later to compute center point of project area
    circle = np.concatenate((expanded_hull_points, expanded_hull_points[0][None, :]), axis=0)
    external_delimitation = LineString(circle)
    for road in sections_df['geometry']:
        start_point = Point(road.coords[0])
        end_point = Point(road.coords[-1])

        if external_delimitation.distance(start_point) < external_delimitation.distance(end_point):
            # start is external centroid
            external_centroid_nodes.append(start_point)
            internal_centroid_nodes.append(end_point)
        else:
            # end is external centroid
            external_centroid_nodes.append(end_point)
            internal_centroid_nodes.append(start_point)
            
    return external_centroid_nodes, internal_centroid_nodes

def create_mesh(expanded_hull_points, project_delimitation, mesh_density=0.001, testing=False, sample_size=500):
    ''' 
    Create mesh of points.
    
    Note: mesh_density 0.001 creates 2 million points
    '''
    x_min, x_max = np.min(expanded_hull_points[:, 0]), np.max(expanded_hull_points[:, 0])
    y_min, y_max = np.min(expanded_hull_points[:, 1]), np.max(expanded_hull_points[:, 1])
    x, y = np.meshgrid(np.arange(x_min, x_max, mesh_density), np.arange(y_min, y_max, mesh_density))
    x = x.reshape(x.shape[0] * x.shape[1])
    y = y.reshape(y.shape[0] * y.shape[1])
    #mesh_points = list(zip(x, y))
    mesh_points = pd.DataFrame(list(zip(x, y)), columns=['x', 'y']) 
    
    print('created {} mesh points'.format(len(mesh_points)))
    del x, y  # free up memory
    
    # for testing sample mesh points at random and run them
    if testing:
        mesh_points = random.sample(mesh_points, sample_size)
        return mesh_points
    
    print('creating bounds')
    # keep those inside external delimitation and outside project delimitation
    external_delimitation_poly = Polygon(expanded_hull_points)
    project_delimitation_poly = Polygon(project_delimitation)
    external_minus_project = external_delimitation_poly.difference(project_delimitation_poly)
    # bottleneck (iterating over points and using contains method is slow)
    '''     
    def mesh_filter(idx, point, polygons):
        for i in idx.intersection((point.coords[0])):
            if point.intersects(polygons[i]):
                return True
        return False
            
    ext_min_proj_polygons = multisplit_polygon(external_minus_project, 0.005)
    idx = index.Index()
    for pos, poly in enumerate(ext_min_proj_polygons):
        idx.insert(pos, poly.bounds)
        
    mesh_points = list(filter(lambda p: mesh_filter(idx, p, ext_min_proj_polygons), MultiPoint(mesh_points)))
    '''  

    print('pre-filtering (polygon split)')
    ext_min_proj_polygons = multisplit_polygon(external_minus_project, 0.3)
    print('pre-filtering (polygon gdf)', len(ext_min_proj_polygons))
    ext_min_proj_polygons_gdf = gpd.GeoDataFrame(geometry=ext_min_proj_polygons)
    print('pre-filtering (mesh points gdf)')
    mesh_points_gdf = gpd.GeoDataFrame(geometry=gpd.points_from_xy(mesh_points.x, mesh_points.y))
    print('filtering p1')
    sjoin_gdf = gpd.sjoin(mesh_points_gdf, ext_min_proj_polygons_gdf, how='left', op='intersects')
    print('filtering p2')
    mesh_points = list(sjoin_gdf[sjoin_gdf['index_right'].notnull()]['geometry'].values)
    
    print('kept {} mesh points'.format(len(mesh_points)))
    
    return mesh_points


def create_external_taz(dir_taz, sections_df, project_delimitation, output_dir=None):
    """
    3 Steps for Create external TAZs
    1. Create a external demand delimitation:
    - load SFCTA data as Geopandas point (one point = one origin or one destination)
    - Get convex hull of the point
    - Use the convex hull (+ buffer) as the external demand delimitation
    2. create external centroids:
    - select road with no fnode and capacity above 800 from sections_df
    - create a point at the end of all selected road
    - plot the points, get a list of points to remove visually
    3. create external TAZs:
    - create a mesh a points inside the external demand delimitation and outside the internal demand delimitation (project delimitation)
    - use a Direction API (maybe Here direction):
    for every mesh point:
        Query path from mesh point to center of the project area
        Find the closest external centroid to the path. Test that all paths are not to far from existing
            external centroid --> if not, we might be missing one external centroid.
        Associate the external centroid to the mesh point.
        create external TAZ from mesh of points (if you reach point, Theo has already done it for internal TAZs)

    @param dir_taz:         folder containing prefix_fremont_legs.csv where prefix=ending, internal and starting
    @param sections_df:     geo pandas data frame of the aimsun sections
    """
    # 1. Create a external demand delimitation:
    expanded_hull_points = create_external_delim(dir_taz)
    # 2. create external centroids:
    external_centroid_nodes, internal_centroid_nodes = create_external_centroids(sections_df, expanded_hull_points)
    # 3. create external TAZs:
    mesh_points = create_mesh(expanded_hull_points, project_delimitation, 0.01)

    # compute center of project area
    internal_centroid_nodes = np.array([(p.x, p.y) for p in internal_centroid_nodes])
    project_center = np.mean(internal_centroid_nodes, axis=0)
    project_center = Point(project_center[0], project_center[1])

    # for each mesh point find closest external centroid to its query path 
    project_delimitation_line = LineString(project_delimitation + [project_delimitation[0]])   
    distance_to_centroid_threshold = 0.005
    intersection_to_centroid_paths = []
    info_point_to_center = []  # desired result
    
    count = 1
    for point in mesh_points:
        path = shortest_path_by_travel_time(point, project_center)
        
        if count % 1000 == 0:
            print("progress by another 1000!")
        count += 1
        # find intersection (point) of path and project delimitation
        intersect_point = project_delimitation_line.intersection(path)

        # find closest centroid to intersection point
        min_distance = float('inf')
        closest_centroid = None
        for centroid in external_centroid_nodes:
            dist = intersect_point.distance(centroid)
            if dist < min_distance:
                min_distance = dist
                closest_centroid = centroid
        
        #print(closest_centroid)
        
        if min_distance < distance_to_centroid_threshold:
            # path intersection to centroid
            intersection_to_centroid = [(intersect_point.x, intersect_point.y), (closest_centroid.x, closest_centroid.y)]
            intersection_to_centroid_paths.append(LineString(intersection_to_centroid))

            # write result to csv
            info_point_to_center.append([(point.x, point.y), project_center, closest_centroid, min_distance, path])

    #if testing:
        #kepler_map = KeplerGl(height=600)
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_center]}, crs='epsg:4326'), name='project_center')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': external_centroid_nodes}, crs='epsg:4326'), name='external_centroids')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_delimitation_line]}, crs='epsg:4326'), name='project_delimitation')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [external_delimitation]}, crs='epsg:4326'), name='external_delimitation')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': mesh_points}, crs='epsg:4326'), name='mesh_points')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [l[-1] for l in info_point_to_center]}, crs='epsg:4326'), name='paths')
        #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': intersection_to_centroid_paths}, crs='epsg:4326'), name='intersection_to_centroid_paths')
        #file_path = 'mesh_points_to_external_centroids.html'
        #if output_dir:
            #file_path = os.path.join(output_dir, file_path)
        #kepler_map.save_to_html(file_name=file_path)

    #create dataframe
    info_points_col = ['origin_mesh_point','destination','closest_external_centroid','distance_to_centroid','path']
    info_points_df = pd.DataFrame(info_point_to_center, columns = info_points_col)

    # write results to csv
    mesh_points_to_centroid_file_path = 'mesh_point_to_centroid.csv'
    if output_dir:
        mesh_points_to_centroid_file_path = os.path.join(output_dir, mesh_points_to_centroid_file_path)
    to_csv(mesh_points_to_centroid_file_path,
           'origin_mesh_point,destination,closest_external_centroid,distance_to_centroid,path',
           info_point_to_center)
    return info_points_df
    #render    
    

In [11]:
def render_taz_from_csv(csv_file, output_dir=None):
    project_delimitation_line = LineString(project_delimitation + [project_delimitation[0]])
    
    info_points_df = pd.read_csv(csv_file)
    external_centroid_nodes = info_points_df['closest_external_centroid'].map(convert_point_to_coord).tolist()
    external_centroid_nodes = [Point(coord[0], coord[1]) for coord in external_centroid_nodes]
    
    kepler_map = KeplerGl(height=600)
    #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_center]}, crs='epsg:4326'), name='project_center')
    kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': external_centroid_nodes}, crs='epsg:4326'), name='external_centroids')
    kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [project_delimitation_line]}, crs='epsg:4326'), name='project_delimitation')
    #kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': [external_delimitation]}, crs='epsg:4326'), name='external_delimitation')
    
    taz_id = 0
    boundary_list = []
    taz_name_list = []
    external_centroids = info_points_df['closest_external_centroid'].unique()
    for centroid in external_centroids:
        taz_df = info_points_df.loc[info_points_df['closest_external_centroid']==centroid]
        taz_points = np.array(taz_df['origin_mesh_point'].map(convert_point_to_coord).tolist())
        taz_hull = ConvexHull(taz_points)
        taz_boundary = taz_points[taz_hull.vertices, :]
        taz_poly = Polygon(taz_boundary)
        taz_name = 'External TAZ' + str(taz_id)
        kepler_map.add_data(data=gpd.GeoDataFrame({'geometry': taz_poly}, crs='epsg:4326',index=[0]), name=taz_name)
        taz_name_list.append(taz_name)
        boundary_list.append(taz_poly)
        taz_id += 1
    taz_gpd = gpd.GeoDataFrame({'taz_name':taz_name_list,'geometry':boundary_list})
    file_path = 'external_taz.html'
    if output_dir:
        file_path = os.path.join(output_dir, file_path)
    kepler_map.save_to_html(file_name=file_path)
    return taz_gpd

In [13]:
def convert_point_to_coord(string):
    lon, lat = string.split(' ')[1:]
    lon = float(lon.split('(')[1])
    lat = float(lat.split(')')[0])
    return [lon, lat]

In [14]:
# print(sections_shp.columns)
# print(sections_shp.head)
output_dir = os.path.join(data_path, 'Data processing', 'Kepler maps', 'HereAPI')
mesh_points_to_centroid_file_path = 'mesh_point_to_centroid.csv'
taz_gpd = render_taz_from_csv(os.path.join(output_dir, mesh_points_to_centroid_file_path), output_dir=output_dir)

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md
Map saved to C:\Users\jainc\Dropbox/Private Structured data collection\Data processing\Kepler maps\HereAPI\external_taz.html!


In [ ]:
sfcta_folder = os.path.join(data_path, "Data processing", "Raw", "Demand", "OD demand", "SFCTA demand data")
sections_path = os.path.join(data_path, "Aimsun", "Inputs", "sections.shp")

sections_shp = gpd.GeoDataFrame.from_file(sections_path)
sections_shp = sections_shp.to_crs(crs_degree) 
# print(sections_shp.columns)
# print(sections_shp.head)
df_test = create_external_taz(sfcta_folder, sections_shp, project_delimitation)

## To do:
1. Split the function create_external_taz into several subfunction that needs to be run sequentially
    1. Create a external demand delimitation:
    2. Create external centroids:
    3. Create external TAZs:
        1. Create a mesh grid
        2. Associate every mesh grid to an external centroid
        3. From the mesh grid create the external TAZ (Theo can do it)
2. For every sub-function write some code to check the function (or render the output of the function)

In [71]:
from random import uniform

def newpoint():
    return uniform(-180,180), uniform(-90, 90)

points = [newpoint() for x in range(100)]

In [76]:
query_result = table(
                          points,
                          output='df')
#shortest_path = wkt.loads(query_result[0]['geometry'])
#print(query_result['geometry'])
query_result[0]

,0,1,2,3,4,5,6,7,8,9,...,90,91,92,93,94,95,96,97,98,99
0,0.0,13209.9,NaN,NaN,11821.0,NaN,13207.8,NaN,NaN,NaN,...,793397.8,NaN,NaN,692913.6,NaN,1205259.2,13207.8,NaN,NaN,12864.5
1,13210.8,0.0,NaN,NaN,2453.9,NaN,2.1,NaN,NaN,NaN,...,792028.6,NaN,NaN,691544.4,NaN,1203890.0,2.1,NaN,NaN,826.9
2,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,1119.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,14.3,...,NaN,NaN,NaN,NaN,204755.2,NaN,NaN,NaN,NaN,NaN
4,11819.1,2444.4,NaN,NaN,0.0,NaN,2442.3,NaN,NaN,NaN,...,790552.1,NaN,NaN,690067.9,NaN,1202413.5,2442.3,NaN,NaN,2099.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1206094.7,1204801.4,NaN,NaN,1203401.5,NaN,1204799.3,NaN,NaN,NaN,...,828595.2,NaN,NaN,701132.8,NaN,0.0,1204799.3,NaN,NaN,1204456.0
96,13208.7,2.1,NaN,NaN,2451.8,NaN,0.0,NaN,NaN,NaN,...,792026.5,NaN,NaN,691542.3,NaN,1203887.9,0.0,NaN,NaN,824.8
97,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
98,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [73]:
query_result = simple_route(
                          [-121.941339, 37.5241981], [-121.9253273, 37.4977866],
                          output='route', overview="full", geometry='wkt')
shortest_path = wkt.loads(query_result[0]['geometry'])
#print(query_result['geometry'])

In [74]:
gdf = gpd.GeoDataFrame(geometry=[shortest_path])

In [75]:
map_2 = KeplerGl(height=1000)
map_2.add_data(data=gdf, name = "Route")
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Route': {'index': [0], 'columns': ['geometry'], 'data': [['LINESTRING (-121.9413399999999967 3…

In [16]:
split = multisplit_polygon(Polygon(create_external_delim(sfcta_folder)), 0.5)

In [ ]:
nodes_graphs = []
streets_graphs = []
empty_polygons = []
total_time = 0

for idx, s in enumerate(split):
    
    try:
        start = time.time()
        
        osm_graph = ox.graph_from_polygon(s, truncate_by_edge=True)
        osm_graph = ox.add_edge_speeds(osm_graph)
        osm_graph = ox.add_edge_travel_times(osm_graph)
        
        nodes, streets = ox.graph_to_gdfs(osm_graph)
        nodes_graphs.append(nodes)
        streets_graphs.append(streets)
        
        end = time.time()
        
        print("Subsection {} finished. Took {}".format(idx, end - start))
        
        total_time += end - start
        
    except:
        print("{} turned up empty".format(s))
        empty_polygons.append(s)
        continue

In [ ]:
network_all = pd.concat(nodes_graphs)
network_all

In [17]:
gdf = gpd.GeoDataFrame(geometry=split)
map_2 = KeplerGl(height=1000)
map_2.add_data(data=gdf, name = "Split")
map_2

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(data={'Split': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], 'columns': ['geometry'],…